In [1]:
"""カルマンフィルター"""
import sympy
sympy.init_printing()

In [2]:
"""システムの状態遷移(雑音なし)"""
# システム状態n×1行列
n = sympy.symbols('n', integer=True)

# 時刻kにおける、真のシステム状態xk
xk = sympy.MatrixSymbol('xbm_k', n, 1)
xk1 = sympy.MatrixSymbol('xbm_k_-_1', n, 1)

# システムの時間遷移に関する線形モデル
Fk = sympy.MatrixSymbol('F_k', n, n)

# 制御入力
uk = sympy.MatrixSymbol('ubm_k', n, 1)

# システムの状態遷移(雑音なし)
sympy.Eq(xk,Fk*xk1 + uk)

xbmₖ = Fₖ⋅xbmₖ ₋ ₁ + ubmₖ

In [3]:
"""システムの雑音"""
# 時間遷移に関する雑音(process noise)モデルの行列
# 及びその雑音l×1行列
l = sympy.symbols('l', integer=True)
Gk = sympy.MatrixSymbol('G_k', n, l)
wk = sympy.MatrixSymbol('wbm_k', l, 1)

# 雑音は共分散行列Qkかつ零平均の多変数正規分布に従う。
Qk = sympy.MatrixSymbol('Q_k', l, l)
wk,sympy.Function('N')(0,Qk)

(wbmₖ, N(0, Qₖ))

In [4]:
"""システムの状態遷移(雑音あり)"""
sympy.Eq(xk,Fk*xk1 + uk + Gk*wk)

xbmₖ = Fₖ⋅xbmₖ ₋ ₁ + Gₖ⋅wbmₖ + ubmₖ

In [5]:
"""観測方程式(雑音なし)"""
# 観測量m×1行列
m = sympy.symbols('m', integer=True)

# ある時刻kにおける観測量
zk = sympy.MatrixSymbol('zbm_k', m, 1)

# 状態空間を観測空間に線形写像する
# 役割を担う観測モデル
Hk = sympy.MatrixSymbol('H_k', m, n)

# 観測方程式(雑音なし)
sympy.Eq(zk, Hk*xk)

zbmₖ = Hₖ⋅xbmₖ

In [6]:
"""観測の雑音"""
# 観測雑音m×1行列
vk = sympy.MatrixSymbol('vbm_k', m, 1)

# 雑音は共分散行列Rkかつ零平均の多変数正規分布に従う。
Rk = sympy.MatrixSymbol('R_k', m, m)
vk,sympy.Function('N')(0,Rk)

(vbmₖ, N(0, Rₖ))

In [7]:
"""観測方程式(雑音あり)"""
sympy.Eq(zk, Hk*xk + vk)

zbmₖ = Hₖ⋅xbmₖ + vbmₖ

In [8]:
# システムの初期条件と雑音は、互いに統計的に独立であると仮定する。

In [9]:
"""不偏量"""
# 期待値
E = sympy.Function('E')

# 共分散
cov = sympy.Function('cov')

In [10]:
"""予測(システム状態の推定値)"""
# 時刻k-1で推定したシステム状態の推定値
xk1_k1 = sympy.MatrixSymbol('\hat{x}_{k-1|k-1}', n, 1)

# 時刻k-1の推定値から予測した、時刻kのシステム状態の推定値
xk_k1 = sympy.MatrixSymbol('\hat{x}_{k|k-1}', n, 1)
sympy.Eq(xk_k1,Fk*xk1_k1 + uk),\
sympy.Eq(E(xk-xk_k1),0)

(\hat{x}_{k|k-1} = Fₖ⋅\hat{x}_{k-1|k-1} + ubmₖ, E(-\hat{x}_{k|k-1} + xbmₖ) = 0
)

In [11]:
"""予測(システム状態の推定値の精度)"""
# システム状態の誤差の共分散行列(推定値の精度)
# 時刻k-1で推定したシステム状態の推定値の精度
Pk1_k1 = sympy.MatrixSymbol('P_{k-1|k-1}', n, n)

# システムモデルと時刻k-1の精度から予測される、時刻kでのシステム状態の精度
Pk_k1 = sympy.MatrixSymbol('P_{k|k-1}', n, n)
sympy.Eq(Pk_k1,Fk*Pk1_k1*Fk.T + Gk*Qk*Gk.T),\
sympy.Eq(Pk_k1,cov(xk-xk_k1)*sympy.Identity(n))

⎛                             T           T                                   
⎝P_{k|k-1} = Fₖ⋅P_{k-1|k-1}⋅Fₖ  + Gₖ⋅Qₖ⋅Gₖ , P_{k|k-1} = cov(-\hat{x}_{k|k-1} 

         ⎞
+ xbmₖ)⋅I⎠

In [12]:
"""更新(観測残差,innovation)"""
# 観測値と、予測したシステム状態の推定値から
# 計算した予測観測値との差
ek = sympy.MatrixSymbol('ebm_k', m, 1)
sympy.Eq(ek,zk-Hk*xk_k1),\
sympy.Eq(E(ek),0)

(ebmₖ = -Hₖ⋅\hat{x}_{k|k-1} + zbmₖ, E(ebmₖ) = 0)

In [13]:
"""更新(観測残差の共分散)"""
Sk = sympy.MatrixSymbol('S_k', m, m)
sympy.Eq(Sk,Rk+Hk*Pk_k1*Hk.T),\
sympy.Eq(Sk,cov(ek)*sympy.Identity(m))

⎛                    T                       ⎞
⎝Sₖ = Hₖ⋅P_{k|k-1}⋅Hₖ  + Rₖ, Sₖ = cov(ebmₖ)⋅I⎠

In [14]:
"""更新(最適カルマンゲイン)"""
Kk = sympy.MatrixSymbol('K_k', n, m)
sympy.Eq(Kk, Pk_k1*Hk.T*Sk.inverse())

In [15]:
"""観測値によるシステム状態の推定値の更新"""
xk_k = sympy.MatrixSymbol('\hat{x}_{k|k}', n, 1)
sympy.Eq(xk_k, xk_k1 + Kk*ek),\
sympy.Eq(E(xk-xk_k),0)

(\hat{x}_{k|k} = Kₖ⋅ebmₖ + \hat{x}_{k|k-1}, E(-\hat{x}_{k|k} + xbmₖ) = 0)

In [16]:
"""観測値によるシステム状態の推定値の精度の更新"""
Pk_k = sympy.MatrixSymbol('P_{k|k}', n, n)
sympy.Eq(Pk_k, (sympy.Identity(n)-Kk*Hk)*Pk_k1),\
sympy.Eq(Pk_k,cov(xk-xk_k)*sympy.Identity(n))

(P_{k|k} = (I -Kₖ⋅Hₖ)⋅P_{k|k-1}, P_{k|k} = cov(-\hat{x}_{k|k} + xbmₖ)⋅I)